In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import caesar
from astropy.cosmology import LambdaCDM
import astropy.units as u
from astropy.io import fits
from scipy import interpolate
import psutil
import gc
import h5py

# # Function to print RAM usage
# def print_ram_usage():
#     process = psutil.Process()
#     ram_usage = process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB
#     print(f"RAM Usage: {ram_usage:.2f} MB")

%run  guideclass.ipynb
cosmo = LambdaCDM(H0 = 68, Om0= 0.3, Ode0= 0.7, Ob0=0.048)  

sb = simba('PC39BP4', size=100)

# for i in range(100, 150,1):
#     print('Before opening caesar file')
#     print_ram_usage()
#     cs = sb.get_caesar(i, fname=f'm100n1024_%s.hdf5')
#     f = cs.data_file
#     with h5py.File(f, 'r') as hf:
#         tree = np.asarray(hf['tree_data']['progen_galaxy_star'])
#         print(tree)
#     del cs
#     gc.collect()
#     print('After closing caesar file')
#     print_ram_usage()



In [8]:
caesar.progen.z_to_snap(0.1, snaplist_file='Simba')

(145, 0.10247870286765726)

In [3]:
import numpy as np
import h5py
from astropy.io import fits

def read_progen(ids, outname, snaplist, sb):
    # Initialize a dictionary to store progenid data for each snapshot
    progenid_dict = {i: [] for i in snaplist}
    
    # Get GroupIDs and progenid from each snapshot
    for i in np.sort(snaplist):
        cs = sb.get_caesar(i, fname=f'm100n1024_%03d.hdf5' %i)
        allids = np.asarray([galaxy.GroupID for galaxy in cs.galaxies])
        fname = cs.data_file
        
        with h5py.File(fname, 'r') as hf:
            progenid = np.asarray(hf['tree_data']['progen_galaxy_star'])[:, 0]
            # Mask to exclude -1 progenids
            mask = progenid != -1
            progenid = progenid[mask]
            allids = allids[mask]
        
        progenid_dict[i] = np.column_stack((allids, progenid))
    
    # Extract GroupIDs from the first snapshot (snaplist[0] should be 151 if passed in the right order)
    base_snapshot = np.sort(snaplist)[0]
    base_data = progenid_dict[base_snapshot]
    base_groupids = base_data[:, 0]
    
    # Initialize an array to store the result
    progenid_table = np.full((len(base_groupids), len(snaplist) + 1), -1, dtype=int)
    progenid_table[:, 0] = base_groupids
    
    # Fill the table with progenid data for each snapshot
    for i, snap in enumerate(np.sort(snaplist)):
        snap_data = progenid_dict[snap]
        for j, groupid in enumerate(base_groupids):
            idx = np.where(snap_data[:, 0] == groupid)[0]
            if len(idx) > 0:
                progenid_table[j, i + 1] = snap_data[idx, 1]
    
    # Create a FITS table with appropriate columns
    col1 = fits.Column(name='GroupID', format='K', array=progenid_table[:, 0])
    columns = [col1]
    for i, snap in enumerate(np.sort(snaplist)):
        col = fits.Column(name=f'ProgenID_snap{snap}', format='K', array=progenid_table[:, i + 1])
        columns.append(col)
    
    hdu = fits.BinTableHDU.from_columns(columns)
    hdu.writeto(outname, overwrite=True)

# Example usage:
cs = sb.get_caesar(151, fname=f'm100n1024_%s.hdf5')
read_progen(np.asarray([galaxy.GroupID for galaxy in cs.galaxies]), 'output.fits', np.arange(44, 151), sb)


yt : [INFO     ] 2024-07-22 11:09:55,509 Opening /media/lorenzong/Data1/SIMBA_catalogs/m100n1024_151.hdf5
yt : [INFO     ] 2024-07-22 11:09:55,518 Found 464043 halos
yt : [INFO     ] 2024-07-22 11:09:55,521 Found 55609 galaxies
yt : [INFO     ] 2024-07-22 11:09:55,867 Opening /media/lorenzong/Data1/SIMBA_catalogs/m100n1024_044.hdf5
yt : [INFO     ] 2024-07-22 11:09:56,084 Found 304903 halos
yt : [INFO     ] 2024-07-22 11:09:56,386 Found 13162 galaxies
yt : [INFO     ] 2024-07-22 11:09:56,451 Opening /media/lorenzong/Data1/SIMBA_catalogs/m100n1024_045.hdf5
yt : [INFO     ] 2024-07-22 11:09:56,658 Found 320267 halos
yt : [INFO     ] 2024-07-22 11:09:57,110 Found 14221 galaxies
yt : [INFO     ] 2024-07-22 11:09:57,168 Opening /media/lorenzong/Data1/SIMBA_catalogs/m100n1024_046.hdf5
yt : [INFO     ] 2024-07-22 11:09:57,368 Found 335080 halos
yt : [INFO     ] 2024-07-22 11:09:57,733 Found 15089 galaxies
yt : [INFO     ] 2024-07-22 11:09:57,914 Opening /media/lorenzong/Data1/SIMBA_catalogs/m

In [29]:
cs = sb.get_caesar(129, fname=f'm100n1024_%s.hdf5')

yt : [INFO     ] 2024-07-22 09:21:31,508 Opening /media/lorenzong/Data1/SIMBA_catalogs/m100n1024_129.hdf5
yt : [INFO     ] 2024-07-22 09:21:31,519 Found 495012 halos
yt : [INFO     ] 2024-07-22 09:21:31,522 Found 46575 galaxies


In [36]:
redshift = caesar.progen.get_progen_redshift('/media/lorenzong/Data1/SIMBA_catalogs/m100n1024_149.hdf5','descend_galaxy_star')

In [49]:
f = cs.data_file
with h5py.File(f, 'r') as hf:
    tree = np.asarray(hf['tree_data']['descend_galaxy_star'])
    print(tree)

[    0     1     2 ...    -1    -1 43281]


In [40]:
cs.__dir__()

['_ds',
 'data_file',
 'skip_hash_check',
 '_halo_dmlist',
 '_halo_slist',
 '_halo_glist',
 '_halo_bhlist',
 '_halo_dlist',
 '_galaxy_slist',
 '_galaxy_glist',
 '_galaxy_bhlist',
 '_galaxy_dlist',
 '_cloud_glist',
 '_cloud_dlist',
 'hash',
 'caesar',
 'unit_registry',
 'simulation',
 '_galaxy_index_list',
 '_halo_data',
 '_halo_dicts',
 '_progen_halo_dm',
 'nhalos',
 'halos',
 '_galaxy_data',
 '_galaxy_dicts',
 'ngalaxies',
 'galaxies',
 '_cloud_index_list',
 '_progen_galaxy_star',
 '_descend_galaxy_star',
 '_cloud_data',
 '_cloud_dicts',
 'nclouds',
 'clouds',
 '__module__',
 '__init__',
 'yt_dataset',
 'central_galaxies',
 'satellite_galaxies',
 'galinfo',
 'haloinfo',
 'cloudinfo',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__s

In [43]:
test = caesar.progen.collect_group_IDs(cs, 'galaxy', 'PartType4', snap_dir='/media/lorenzong/Data1/simba_hig_res')

Could not determine file type by extension!


SystemExit: 

In [44]:
%tb

SystemExit: 